In [1]:
# ----------------------------------------------------------------------------
# Project     : mrs - multi-turn response selection
# Created By  : Eungis
# Team        : AI Engineering
# Created Date: 2023-11-30
# Updated Date: 2024-02-20
# Purpose     : Make data_loader for loading data
# version     : 0.0.1
# ---------------------------------------------------------------------------

In [2]:
import logging
import pandas as pd
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

DATA_ROOT = "../data/"
LOGGER = logging.getLogger(__name__)

In [3]:
from typing import List
from dataclasses import dataclass


@dataclass
class Session:
    conv: List[str]
    """Conversation: List of utterences"""


class SessionBuilder:
    def __init__(self, style: str):
        self.style = style
        self.logger = self._set_logger()

    def _set_logger(self):
        logger = logging.getLogger(__name__)
        return logger

    def build_sessions(self, data_path: str) -> List[List[str]]:
        # data to load must be separated with `tab`
        data = pd.read_csv(data_path, sep="\t")
        styles = data.columns.tolist()
        if self.style not in styles:
            raise ValueError(
                f"Unsupported style. Style must be one of {styles}.\nInput: {self.style}"
            )

        # use specified style conversational data
        data = data[[self.style]]
        data["group"] = data[self.style].isnull().cumsum()
        n_sessions = data["group"].iat[-1] + 1
        self.logger.debug(f"Number of sessions: {n_sessions}")

        # split data into sessions
        sessions: List[Session] = []
        groups = data.groupby("group", as_index=False, group_keys=False)

        for i, group in groups:
            session = group.dropna()[self.style].tolist()
            sessions += [Session(conv=session)]

        assert n_sessions == len(sessions)
        return sessions

    def build_short_sessions(
        self, sessions: List[Session], ctx_len: int = 4
    ) -> List[Session]:
        short_sessions = []
        for session in sessions:
            for i in range(len(session.conv) - ctx_len + 1):
                short_sessions.append(Session(conv=session.conv[i : i + ctx_len]))
        return short_sessions

    def get_utterances(self, sessions: List[Session]):
        all_utts = set()
        for session in sessions:
            for utt in session.conv:
                all_utts.add(utt)
        return list(all_utts)

In [4]:
from transformers import AutoTokenizer

builder = SessionBuilder(style="formal")
sessions = builder.build_sessions(data_path=DATA_ROOT + "smilestyle_dataset.tsv")
short_sessions = builder.build_short_sessions(sessions, ctx_len=4)
utts = builder.get_utterances(sessions)

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
LOGGER.debug(f"Special tokens: {tokenizer.special_tokens_map}")
LOGGER.debug(f"EOS token & SEP token: {tokenizer.eos_token} / {tokenizer.sep_token}")
special_tokens = {"sep_token": "<SEP>"}
tokenizer.add_special_tokens(special_tokens)

LOGGER.info(
    f"""Special tokens map: {tokenizer.special_tokens_map}
{tokenizer.eos_token}: {tokenizer.eos_token_id}
{tokenizer.sep_token}: {tokenizer.sep_token_id}
{tokenizer.mask_token}: {tokenizer.mask_token_id}"""
)

Number of sessions: 236
Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
EOS token & SEP token: [SEP] / [SEP]
Special tokens map: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '<SEP>', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
[SEP]: 2
<SEP>: 32000
[MASK]: 4


In [5]:
import torch
import random
import logging
import numpy as np
from typing import List, Dict
from torch.utils.data import Dataset
from transformers import AutoTokenizer

MODEL_NAME = "klue/roberta-base"
DATA_ROOT = "../data/"
DATA_PATH = DATA_ROOT + "smilestyle_dataset.tsv"


class PostDataset(Dataset):
    def __init__(self, builder: SessionBuilder):
        # set logger
        self.logger = self._set_logger()

        # set tokenizer
        self.tokenizer = self._set_tokenizer()

        # build sessions
        self.sessions = builder.build_sessions(data_path=DATA_PATH)
        self.short_sessions = builder.build_short_sessions(self.sessions, ctx_len=4)
        self.utts = builder.get_utterances(self.sessions)

    def _set_logger(self):
        logger = logging.getLogger(__name__)
        return logger

    def _set_tokenizer(self):
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        special_tokens = {"sep_token": "<SEP>"}
        tokenizer.add_special_tokens(special_tokens)
        return tokenizer

    def _mask_tokens(self, tokens: List, ratio: float = 0.15) -> List:
        tokens = np.array(tokens)
        n_mask = int(len(tokens) * ratio)
        mask_pos = random.sample(range(len(tokens)), n_mask)

        # fancy indexing
        tokens[mask_pos] = self.tokenizer.mask_token_id
        tokens = tokens.tolist()
        return tokens

    def _get_mask_positions(self, tokens: List) -> List:
        tokens = np.array(tokens)
        mask_positions = np.where(tokens == self.tokenizer.mask_token_id)[0].tolist()
        return mask_positions

    def construct_mlm_inputs(self, short_session: Session) -> Dict[str, list]:
        corrupt_tokens, output_tokens = [], []

        for i, utt in enumerate(short_session.conv):
            tokens = self.tokenizer.encode(utt, add_special_tokens=False)
            masked_tokens = self._mask_tokens(tokens)

            if i == len(short_session.conv) - 1:
                output_tokens.extend(tokens)
                corrupt_tokens.extend(masked_tokens)
            else:
                output_tokens.extend(tokens + [self.tokenizer.sep_token_id])
                corrupt_tokens.extend(masked_tokens + [self.tokenizer.sep_token_id])

        corrupt_mask_positions = self._get_mask_positions(corrupt_tokens)
        return_value = {
            "output_tokens": output_tokens,
            "corrupt_tokens": corrupt_tokens,
            "corrupt_mask_positions": corrupt_mask_positions,
        }

        return return_value

    def construct_urc_inputs(self, short_session: Session) -> Dict[str, List]:
        urc_tokens, ctx_utts = [], []

        for i in range(len(short_session.conv)):
            utt = short_session.conv[i]
            tokens = self.tokenizer.encode(utt, add_special_tokens=False)

            if i == len(short_session.conv) - 1:
                urc_tokens += [self.tokenizer.eos_token_id]
                positive_tokens = [self.tokenizer.cls_token_id] + urc_tokens + tokens

                while True:
                    random_neg_response = random.choice(self.utts)
                    if random_neg_response not in ctx_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(
                    random_neg_response, add_special_tokens=False
                )
                random_tokens = (
                    [self.tokenizer.cls_token_id]
                    + urc_tokens
                    + random_neg_response_token
                )
                ctx_neg_response = random.choice(ctx_utts)
                ctx_neg_response_token = self.tokenizer.encode(
                    ctx_neg_response, add_special_tokens=False
                )
                ctx_neg_tokens = (
                    [self.tokenizer.cls_token_id] + urc_tokens + ctx_neg_response_token
                )
            else:
                urc_tokens += tokens + [self.tokenizer.sep_token_id]

            ctx_utts.append(utt)

        return_value = {
            "positive_tokens": positive_tokens,
            "random_negative_tokens": random_tokens,
            "context_negative_tokens": ctx_neg_tokens,
            "urc_labels": [0, 1, 2],
        }
        return return_value

    def __len__(self):
        return len(self.short_sessions)

    def __getitem__(self, idx):
        # ---- input data for MLM ---- #
        short_session = self.short_sessions[idx]
        mlm_input = self.construct_mlm_inputs(short_session)

        # ---- intput data for utterance relevance classification ---- #
        urc_input = self.construct_urc_inputs(short_session)

        return_value = dict()
        return_value["mlm_input"] = mlm_input
        return_value["urc_input"] = urc_input

        return return_value

In [6]:
builder = SessionBuilder(style="formal")
post_dataset = PostDataset(builder)

https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Number of sessions: 236


In [7]:
sample = post_dataset[0]
print(post_dataset.tokenizer.decode(sample["mlm_input"]["output_tokens"]))
print(post_dataset.tokenizer.decode(sample["mlm_input"]["corrupt_tokens"]))
print(sample["mlm_input"]["corrupt_mask_positions"])
print(post_dataset.tokenizer.decode(sample["urc_input"]["positive_tokens"]))
print(post_dataset.tokenizer.decode(sample["urc_input"]["random_negative_tokens"]))
print(post_dataset.tokenizer.decode(sample["urc_input"]["context_negative_tokens"]))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
안녕하 [MASK]. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요 [MASK] 키우는거 안 힘드세요 [MASK] <SEP> 제가 워낙 [MASK]를 좋아해서 [MASK]게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요 [MASK]
[2, 19, 26, 31, 34, 52]
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 가장 나이가 많은 고양이가 어떻게 돼요?
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 저는 캘리포니아에서 왔어요. 어떤 일을 하시나요?
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 고양이를 6마리나요? 키우는거 안 힘드세요?


In [45]:
from typing import List
from torch.nn.utils.rnn import pad_sequence


class PostDatasetCollator:
    def __init__(self, pad_idx: int, max_length: int):
        self.pad_idx = pad_idx
        self.max_length = max_length

    def __call__(self, batch: List[dict]):
        # |batch| = [
        # {
        # 'mlm_input': {
        # 'output_tokens': list(),
        # 'corrupt_tokens': list(),
        # 'corrupt_mask_positions': list()
        # },
        # 'urc_input': {
        # 'positive_tokens': list(),
        # 'random_negative_tokens': list(),
        # 'context_negative_tokens': list(),
        # 'urc_labels': list()
        # }
        # }, ...
        # ]

        # initialize batch output bags
        mlm_output_tokens_inputs, mlm_corrupt_tokens_inputs = [], []
        urc_inputs = []
        mlm_corrupt_mask_positions, urc_labels = [], []

        for sample in batch:
            mlm_input, urc_input = sample["mlm_input"], sample["urc_input"]

            mlm_output_tokens_inputs.append(
                torch.tensor(mlm_input["output_tokens"][: self.max_length])
            )
            mlm_corrupt_tokens_inputs.append(
                torch.tensor(mlm_input["corrupt_tokens"][: self.max_length])
            )
            mlm_corrupt_mask_positions.append(
                torch.tensor(mlm_input["corrupt_mask_positions"])
            )

            urc_inputs.append(
                torch.tensor(urc_input["positive_tokens"][: self.max_length])
            )
            urc_inputs.append(
                torch.tensor(urc_input["random_negative_tokens"][: self.max_length])
            )
            urc_inputs.append(
                torch.tensor(urc_input["context_negative_tokens"][: self.max_length])
            )
            urc_labels.append(torch.tensor(urc_input["urc_labels"]))

        # pad sequence
        mlm_output_tokens_inputs = pad_sequence(
            mlm_output_tokens_inputs, batch_first=True, padding_value=self.pad_idx
        )
        mlm_corrupt_tokens_inputs = pad_sequence(
            mlm_corrupt_tokens_inputs, batch_first=True, padding_value=self.pad_idx
        )

        urc_inputs = pad_sequence(
            urc_inputs, batch_first=True, padding_value=self.pad_idx
        )

        # get attention masking positions
        mlm_attentions = (mlm_output_tokens_inputs != self.pad_idx).long()
        urc_attentions = (urc_inputs != self.pad_idx).long()

        return_value = {
            "mlm_inputs": {
                "output_tokens": mlm_output_tokens_inputs,
                "corrupt_tokens": mlm_corrupt_tokens_inputs,
                "mask_positions": mlm_corrupt_mask_positions,
                "attention_masks": mlm_attentions,
            },
            "urc_inputs": {
                "input_tokens": urc_inputs,
                "labels": urc_labels,
                "attention_masks": urc_attentions,
            },
        }
        return return_value

In [46]:
from torch.utils.data import DataLoader

post_dataloader = DataLoader(
    post_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=PostDatasetCollator(
        pad_idx=post_dataset.tokenizer.pad_token_id, max_length=99999
    ),
)

In [47]:
sample_batch = next(iter(post_dataloader))

In [48]:
sample_mlm_inputs = sample_batch["mlm_inputs"]
sample_urc_inputs = sample_batch["urc_inputs"]

In [49]:
sample_mlm_inputs["output_tokens"].shape, sample_mlm_inputs["corrupt_tokens"].shape

(torch.Size([2, 86]), torch.Size([2, 86]))

In [51]:
post_dataset.tokenizer.decode(
    sample_mlm_inputs["output_tokens"][0, :]
), post_dataset.tokenizer.decode(sample_mlm_inputs["corrupt_tokens"][0, :])

('로봇이 능동적으로 사람을 속이려고 할 수 있다는 말씀이신가요? <SEP> 예를 들면 두 명의 사람이 있고, 한명을 살리기 위해 한명을 죽여야 한다면, 로봇은 사람에게 해를 끼칠 수 있지 않을까요? <SEP> 제 생각에는 가만히 있을 것 같아요. <SEP> 제 생각에는 가만히 있지 않고 해를 끼칠수도 있다고 봅니다.',
 '로봇이 능동적으로 사람 [MASK] 속이 [MASK] 할 수 있다는 말씀이신가요? <SEP> [MASK]를 들면 두 명의 [MASK]이 있고, 한명을 살리기 위해 [MASK] [MASK] 죽여야 한다면, 로봇은 사람에게 해를 끼칠 [MASK] 있지 않을까요? <SEP> 제 생각에는 가만히 [MASK]을 것 같아요. <SEP> [MASK] 생각에는 가만히 있지 않고 해를 [MASK]수도 있다고 봅니다.')

In [52]:
post_dataset.tokenizer.decode(
    sample_mlm_inputs["output_tokens"][1, :]
), post_dataset.tokenizer.decode(sample_mlm_inputs["corrupt_tokens"][1, :])

('네, 자이언트 팬입니다. <SEP> 자이언트 팀은 어디 팀인가요? <SEP> 신생 팀인데, 아리조나 대학교 팀입니다. <SEP> 아 그렇군요, 저는 캘리포니아 호크를 응원하고 있습니다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '네, 자이언 [MASK] 팬입니다. <SEP> 자이언트 [MASK]은 어디 팀인가요? <SEP> 신생 팀인데, 아리조나 대학교 [MASK]입니다. <SEP> 아 그렇군 [MASK], 저 [MASK] 캘리포니아 호크를 응원하고 있습니다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]')

In [50]:
sample_mlm_inputs["mask_positions"], sample_mlm_inputs["attention_masks"]

([tensor([ 5,  7, 18, 24, 34, 35, 48, 61, 69, 80]),
  tensor([ 3, 10, 27, 34, 37])],
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [61]:
post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][0]
), post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][1]
), post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][2]
), post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][3]
), post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][4]
), post_dataset.tokenizer.decode(
    sample_urc_inputs["input_tokens"][5]
)

('[CLS] 로봇이 능동적으로 사람을 속이려고 할 수 있다는 말씀이신가요? <SEP> 예를 들면 두 명의 사람이 있고, 한명을 살리기 위해 한명을 죽여야 한다면, 로봇은 사람에게 해를 끼칠 수 있지 않을까요? <SEP> 제 생각에는 가만히 있을 것 같아요. <SEP> [SEP] 제 생각에는 가만히 있지 않고 해를 끼칠수도 있다고 봅니다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 로봇이 능동적으로 사람을 속이려고 할 수 있다는 말씀이신가요? <SEP> 예를 들면 두 명의 사람이 있고, 한명을 살리기 위해 한명을 죽여야 한다면, 로봇은 사람에게 해를 끼칠 수 있지 않을까요? <SEP> 제 생각에는 가만히 있을 것 같아요. <SEP> [SEP] 어떤 의문이요? [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 로봇이 능동적으로 사람을 속이려고 할 수 있다는 말씀이신가요? <SEP> 예를 들면 두 명의 사람이 있고, 한명을 살리기 위해 한명을 죽여야 한다면, 로봇은 사람에게 해를 끼칠 수 있지 않을까요? <SEP> 제 생각에는 가만히 있을 것 같아요. <SEP> [SEP] 예를 들면 두 명의 사람이 있고, 한명을 살리기 위해 한명을 죽여야 한다면, 로봇은 사람에게 해를 끼칠 수 있지 않을까요?',
 '[CLS] 네, 자이언트 팬입니다. <SEP> 자이언트 팀은 어디 팀인가요? <SEP> 신생 팀인데, 아리조나 대학교 팀입니다. <SEP> [SEP] 아 그렇군요, 저는 캘리

In [59]:
sample_urc_inputs["labels"]

[tensor([0, 1, 2]), tensor([0, 1, 2])]

In [62]:
sample_urc_inputs["attention_masks"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1,